In [1]:
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
# from src.wilson import * 
from src.models import * 

## Parameters

In [2]:
L = 4

note hilbert space is on edges
there is no natural ordering to the hilbert space so we will have to impose one.
this is specified by wilson.ordered_edges.
as long as we respect this ordering across all implementations, things should work. Convention is vertical links first.

In [3]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 32, LxLx2 = 32


$$
H = -K \sum_{\square} \prod_{ij \in \square} \sigma_{ij}^z - g \prod_{ij} \sigma_{ij}^x
$$

In [19]:
K=1
g_ham = 1
h = nk.operator.LocalOperator(hi)
for edge in g.edges():
    i, j = edge
    h += -K * nk.operator.spin.sigmaz(hi, i) * nk.operator.spin.sigmaz(hi, j)
for i in range(g.n_nodes):
    h += -g_ham * nk.operator.spin.sigmax(hi, i)

print(h)

LocalOperator(dim=32, #acting_on=48 locations, constant=0.0, dtype=float64)


In [26]:
genn = GENN(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=genn, n_samples=50)
optimizer = nk.optimizer.Sgd(learning_rate=0.1)
preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
try:
    vmc.run(n_iter=250, out=log)
except:
    print(log)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=50 (50 per device/MPI rank) does not divide n_chains=16, increased to 64 (64 per device/MPI rank)
  self.n_samples = n_samples
  0%|          | 0/250 [00:00<?, ?it/s]

None
